# Molecular Music: Sonifying Protein Binding Data

This notebook implements a comprehensive sonification system for molecular biology that translates protein binding characteristics into musical parameters.

## Overview
- **Binding Affinity (Kd)** → **Pitch/Frequency**
- **Binding Kinetics** → **Temporal Parameters**
- **Protein Structure** → **Timbral Characteristics**
- **Molecular Weight** → **Amplitude/Volume**
- **Binding Cooperativity** → **Harmonic Relationships**

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pythonosc import udp_client
from pythonosc.osc_message_builder import OscMessageBuilder
import requests
import json
import time
import warnings
warnings.filterwarnings("ignore")

print("Libraries imported successfully!")

## 1. Data Access Functions

We'll implement functions to access molecular data from public databases:
- **ChEMBL**: Bioactivity database with binding affinities
- **BindingDB**: Database of measured binding affinities  
- **PDB**: 3D protein structures
- **UniProt**: Protein sequences and functional annotations

In [ ]:
class MolecularDataAccess:
    """Class for accessing molecular data from public databases"""
    
    def __init__(self):
        self.chembl_base_url = "https://www.ebi.ac.uk/chembl/api/data"
        self.bindingdb_base_url = "https://www.bindingdb.org/axis2/services/BDBService"
        
    def get_chembl_activities(self, target_chembl_id, limit=100):
        """
        Fetch binding activities from ChEMBL for a specific target
        
        Args:
            target_chembl_id (str): ChEMBL target ID (e.g., 'CHEMBL279')
            limit (int): Maximum number of activities to retrieve
            
        Returns:
            pandas.DataFrame: Binding activity data
        """
        try:
            url = f"{self.chembl_base_url}/activity.json"
            params = {
                'target_chembl_id': target_chembl_id,
                'limit': limit,
                'format': 'json'
            }
            
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            data = response.json()
            activities = data.get('activities', [])
            
            # Convert to DataFrame
            df = pd.DataFrame(activities)
            
            # Filter for binding data
            binding_types = ['Kd', 'Ki', 'IC50', 'EC50']
            if 'standard_type' in df.columns:
                df = df[df['standard_type'].isin(binding_types)]
            
            return df
            
        except Exception as e:
            print(f"Error fetching ChEMBL data: {e}")
            return pd.DataFrame()
    
    def get_sample_binding_data(self):
        """
        Generate sample binding data for demonstration purposes
        """
        np.random.seed(42)
        n_compounds = 50
        
        data = {
            'compound_id': [f'COMP_{i:03d}' for i in range(n_compounds)],
            'kd_nm': np.random.lognormal(mean=3, sigma=2, size=n_compounds),  # Kd in nM
            'kon_m1s1': np.random.lognormal(mean=12, sigma=1, size=n_compounds),  # kon in M-1s-1
            'koff_s1': np.random.lognormal(mean=-2, sigma=1, size=n_compounds),  # koff in s-1
            'molecular_weight': np.random.normal(400, 150, n_compounds),  # MW in Da
            'structure_type': np.random.choice(['alpha_helix', 'beta_sheet', 'random_coil'], n_compounds),
            'cooperativity': np.random.choice(['positive', 'negative', 'none'], n_compounds, p=[0.2, 0.1, 0.7])
        }
        
        df = pd.DataFrame(data)
        df['kd_nm'] = np.clip(df['kd_nm'], 0.1, 10000)  # Realistic Kd range
        df['molecular_weight'] = np.clip(df['molecular_weight'], 100, 1000)  # Realistic MW range
        
        return df

# Initialize data access
data_access = MolecularDataAccess()
print("Molecular data access initialized!")

## 2. Parameter Mapping Functions

These functions translate molecular parameters into musical parameters according to the mappings defined in the README.

In [ ]:
class MolecularToMusicalMapper:
    """Class for mapping molecular parameters to musical parameters"""
    
    def __init__(self):
        # Musical parameter ranges
        self.freq_range = (50, 2000)  # Hz
        self.attack_range = (0.01, 2.0)  # seconds
        self.release_range = (0.1, 5.0)  # seconds
        self.amplitude_range = (0.1, 1.0)  # normalized
        
    def kd_to_frequency(self, kd_values):
        """
        Map Kd values to frequencies
        Low Kd (strong binding) → Lower frequencies (stable, foundational)
        High Kd (weak binding) → Higher frequencies (unstable, transient)
        
        Args:
            kd_values (array-like): Kd values in nM
            
        Returns:
            array: Frequencies in Hz
        """
        kd_array = np.array(kd_values)
        
        # Log scale mapping (Kd spans orders of magnitude)
        log_kd = np.log10(np.clip(kd_array, 0.1, 10000))
        log_kd_norm = (log_kd - np.log10(0.1)) / (np.log10(10000) - np.log10(0.1))
        
        # Invert so low Kd = low frequency
        freq_norm = 1 - log_kd_norm
        
        # Map to frequency range
        frequencies = self.freq_range[0] + freq_norm * (self.freq_range[1] - self.freq_range[0])
        
        return frequencies
    
    def kinetics_to_temporal(self, kon_values, koff_values):
        """
        Map binding kinetics to temporal parameters
        kon (association rate) → Attack time (how quickly sound begins)
        koff (dissociation rate) → Release time (how quickly sound fades)
        
        Args:
            kon_values (array-like): Association rates in M-1s-1
            koff_values (array-like): Dissociation rates in s-1
            
        Returns:
            tuple: (attack_times, release_times) in seconds
        """
        kon_array = np.array(kon_values)
        koff_array = np.array(koff_values)
        
        # Log scale normalization
        log_kon = np.log10(np.clip(kon_array, 1e3, 1e8))
        log_koff = np.log10(np.clip(koff_array, 1e-4, 1e2))
        
        # Normalize to [0, 1]
        kon_norm = (log_kon - 3) / (8 - 3)  # 1e3 to 1e8 range
        koff_norm = (log_koff - (-4)) / (2 - (-4))  # 1e-4 to 1e2 range
        
        # Invert: fast kinetics = short times
        attack_norm = 1 - kon_norm
        release_norm = 1 - koff_norm
        
        # Map to time ranges
        attack_times = self.attack_range[0] + attack_norm * (self.attack_range[1] - self.attack_range[0])
        release_times = self.release_range[0] + release_norm * (self.release_range[1] - self.release_range[0])
        
        return attack_times, release_times
    
    def structure_to_waveform(self, structure_types):
        """
        Map protein structure to waveform types
        α-helices → Sine waves (smooth, periodic structure)
        β-sheets → Sawtooth waves (structured, angular geometry)  
        Random coils → Noise (chaotic, unstructured regions)
        
        Args:
            structure_types (array-like): Structure type strings
            
        Returns:
            array: Waveform type indices (0=sine, 1=sawtooth, 2=noise)
        """
        structure_map = {
            'alpha_helix': 0,
            'beta_sheet': 1, 
            'random_coil': 2
        }
        
        return np.array([structure_map.get(s, 0) for s in structure_types])
    
    def molecular_weight_to_amplitude(self, mw_values):
        """
        Map molecular weight to amplitude
        Larger proteins → Higher amplitude (more molecular mass = more presence)
        
        Args:
            mw_values (array-like): Molecular weights in Da
            
        Returns:
            array: Amplitude values (normalized)
        """
        mw_array = np.array(mw_values)
        
        # Normalize molecular weight
        mw_norm = (mw_array - 100) / (1000 - 100)  # 100-1000 Da range
        mw_norm = np.clip(mw_norm, 0, 1)
        
        # Map to amplitude range
        amplitudes = self.amplitude_range[0] + mw_norm * (self.amplitude_range[1] - self.amplitude_range[0])
        
        return amplitudes
    
    def cooperativity_to_harmony(self, cooperativity_types):
        """
        Map binding cooperativity to harmonic relationships
        Positive cooperativity → Consonant harmonies (perfect 5ths, octaves)
        Negative cooperativity → Dissonant intervals (minor 2nds, tritones)
        No cooperativity → Unison or simple intervals
        
        Args:
            cooperativity_types (array-like): Cooperativity type strings
            
        Returns:
            array: Harmonic interval ratios
        """
        harmony_map = {
            'positive': [1.0, 1.5, 2.0],  # Unison, perfect 5th, octave
            'negative': [1.0, 1.059, 1.414],  # Unison, minor 2nd, tritone
            'none': [1.0, 1.125, 1.25]  # Unison, major 2nd, minor 3rd
        }
        
        harmonies = []
        for coop_type in cooperativity_types:
            harmonies.append(harmony_map.get(coop_type, harmony_map['none']))
        
        return harmonies

# Initialize mapper
mapper = MolecularToMusicalMapper()
print("Molecular to musical mapper initialized!")

## 3. SuperCollider Integration

This section handles communication with SuperCollider via external scripts. The SuperCollider code is kept separate in the `supercollider_scripts/` directory.

In [ ]:
import subprocess
import os
import csv

class SuperColliderInterface:
    """Interface for calling SuperCollider scripts from Python"""
    
    def __init__(self, script_dir="supercollider_scripts"):
        """
        Initialize SuperCollider interface
        
        Args:
            script_dir (str): Directory containing SuperCollider scripts
        """
        self.script_dir = script_dir
        self.sclang_cmd = "sclang"  # Assumes sclang is in PATH
        
    def check_sclang_available(self):
        """Check if sclang is available in the system PATH"""
        try:
            result = subprocess.run([self.sclang_cmd, "--version"], 
                                  capture_output=True, text=True, timeout=5)
            return result.returncode == 0
        except (subprocess.TimeoutExpired, FileNotFoundError):
            return False
    
    def play_binding_event(self, kd, kon, koff, structure, mw, amp=0.3):
        """
        Play a protein binding event using SuperCollider
        
        Args:
            kd (float): Dissociation constant in nM
            kon (float): Association rate in M-1s-1
            koff (float): Dissociation rate in s-1
            structure (int): Structure type (0=sine, 1=sawtooth, 2=noise)
            mw (float): Molecular weight in Da
            amp (float): Amplitude (0-1)
        """
        script_path = os.path.join(self.script_dir, "play_binding_event.scd")
        
        cmd = [
            self.sclang_cmd,
            script_path,
            str(kd), str(kon), str(koff), str(structure), str(mw), str(amp)
        ]
        
        try:
            print(f"Playing binding event: Kd={kd:.2f}nM, structure={structure}")
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
            
            if result.returncode != 0:
                print(f"SuperCollider error: {result.stderr}")
            else:
                print("Sound played successfully!")
                
        except subprocess.TimeoutExpired:
            print("SuperCollider timeout - sound may still be playing")
        except Exception as e:
            print(f"Error running SuperCollider: {e}")
    
    def play_simple_binding(self, freq, amp, attack, release, waveform):
        """
        Play a simple binding event with pre-mapped parameters
        
        Args:
            freq (float): Frequency in Hz
            amp (float): Amplitude (0-1)
            attack (float): Attack time in seconds
            release (float): Release time in seconds
            waveform (int): Waveform type (0=sine, 1=sawtooth, 2=noise)
        """
        script_path = os.path.join(self.script_dir, "play_simple_binding.scd")
        
        cmd = [
            self.sclang_cmd,
            script_path,
            str(freq), str(amp), str(attack), str(release), str(waveform)
        ]
        
        try:
            print(f"Playing simple binding: {freq:.1f}Hz, waveform={waveform}")
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
            
            if result.returncode != 0:
                print(f"SuperCollider error: {result.stderr}")
                
        except Exception as e:
            print(f"Error running SuperCollider: {e}")
    
    def play_sequence_from_data(self, frequencies, amplitudes, attacks, releases, waveforms, delay=0.5):
        """
        Play a sequence of binding events from arrays of parameters
        
        Args:
            frequencies (array): Frequencies in Hz
            amplitudes (array): Amplitudes (0-1)
            attacks (array): Attack times in seconds
            releases (array): Release times in seconds
            waveforms (array): Waveform types
            delay (float): Delay between events in seconds
        """
        # Create temporary CSV file
        csv_filename = "temp_binding_sequence.csv"
        
        with open(csv_filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['freq', 'amp', 'attack', 'release', 'waveform'])
            
            for i in range(len(frequencies)):
                writer.writerow([
                    frequencies[i], amplitudes[i], attacks[i], 
                    releases[i], waveforms[i]
                ])
        
        # Play the sequence
        script_path = os.path.join(self.script_dir, "play_sequence.scd")
        cmd = [self.sclang_cmd, script_path, csv_filename, str(delay)]
        
        try:
            print(f"Playing sequence of {len(frequencies)} binding events...")
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
            
            if result.returncode != 0:
                print(f"SuperCollider error: {result.stderr}")
            else:
                print("Sequence started successfully!")
                
        except Exception as e:
            print(f"Error running SuperCollider: {e}")
        finally:
            # Clean up temporary file
            if os.path.exists(csv_filename):
                os.remove(csv_filename)

# Initialize SuperCollider interface
sc_interface = SuperColliderInterface()

# Check if SuperCollider is available
if sc_interface.check_sclang_available():
    print("SuperCollider (sclang) is available!")
    print("SuperCollider interface initialized successfully!")
else:
    print("WARNING: SuperCollider (sclang) not found in PATH")
    print("Please ensure SuperCollider is installed and sclang is accessible")
    print("You can still run the parameter mapping and visualization functions")

## 4. Phase 1: Proof of Concept Implementation

Let's implement a simple protein-ligand interaction sonification as described in the README.

In [ ]:
# Generate sample binding data
binding_data = data_access.get_sample_binding_data()
print("Generated sample binding data:")
print(f"Number of compounds: {len(binding_data)}")
print("\\nFirst 5 compounds:")
display(binding_data.head())

# Basic statistics
print("\\nData statistics:")
print(f"Kd range: {binding_data['kd_nm'].min():.2f} - {binding_data['kd_nm'].max():.2f} nM")
print(f"MW range: {binding_data['molecular_weight'].min():.1f} - {binding_data['molecular_weight'].max():.1f} Da")
print(f"Structure types: {binding_data['structure_type'].value_counts().to_dict()}")
print(f"Cooperativity types: {binding_data['cooperativity'].value_counts().to_dict()}")

In [ ]:
# Apply parameter mappings to the binding data
frequencies = mapper.kd_to_frequency(binding_data['kd_nm'])
attacks, releases = mapper.kinetics_to_temporal(binding_data['kon_m1s1'], binding_data['koff_s1'])
waveforms = mapper.structure_to_waveform(binding_data['structure_type'])
amplitudes = mapper.molecular_weight_to_amplitude(binding_data['molecular_weight'])

# Add mapped parameters to dataframe
binding_data['frequency_hz'] = frequencies
binding_data['attack_time'] = attacks
binding_data['release_time'] = releases
binding_data['waveform_type'] = waveforms
binding_data['amplitude'] = amplitudes

print("Parameter mappings applied successfully!")
print("\\nMapped parameters for first 5 compounds:")
display(binding_data[['compound_id', 'kd_nm', 'frequency_hz', 'structure_type', 'waveform_type', 'amplitude']].head())

## 5. Visualization and Analysis

Let's visualize the parameter mappings to understand the sonification better.

In [ ]:
# Create visualization plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Molecular to Musical Parameter Mappings', fontsize=16)

# 1. Kd vs Frequency
axes[0,0].scatter(binding_data['kd_nm'], binding_data['frequency_hz'], 
                 c=binding_data['waveform_type'], cmap='viridis', alpha=0.7)
axes[0,0].set_xlabel('Kd (nM)')
axes[0,0].set_ylabel('Frequency (Hz)')
axes[0,0].set_title('Binding Affinity → Pitch')
axes[0,0].set_xscale('log')

# 2. Molecular Weight vs Amplitude
axes[0,1].scatter(binding_data['molecular_weight'], binding_data['amplitude'], 
                 c=binding_data['waveform_type'], cmap='viridis', alpha=0.7)
axes[0,1].set_xlabel('Molecular Weight (Da)')
axes[0,1].set_ylabel('Amplitude')
axes[0,1].set_title('Molecular Weight → Volume')

# 3. Structure Type Distribution
structure_counts = binding_data['structure_type'].value_counts()
axes[0,2].pie(structure_counts.values, labels=structure_counts.index, autopct='%1.1f%%')
axes[0,2].set_title('Protein Structure Distribution')

# 4. Kinetics vs Temporal Parameters
axes[1,0].scatter(binding_data['kon_m1s1'], binding_data['attack_time'], 
                 c=binding_data['waveform_type'], cmap='viridis', alpha=0.7)
axes[1,0].set_xlabel('kon (M⁻¹s⁻¹)')
axes[1,0].set_ylabel('Attack Time (s)')
axes[1,0].set_title('Association Rate → Attack')
axes[1,0].set_xscale('log')

# 5. Release time vs koff
axes[1,1].scatter(binding_data['koff_s1'], binding_data['release_time'], 
                 c=binding_data['waveform_type'], cmap='viridis', alpha=0.7)
axes[1,1].set_xlabel('koff (s⁻¹)')
axes[1,1].set_ylabel('Release Time (s)')
axes[1,1].set_title('Dissociation Rate → Release')
axes[1,1].set_xscale('log')

# 6. Frequency distribution by structure
for i, structure in enumerate(['alpha_helix', 'beta_sheet', 'random_coil']):
    subset = binding_data[binding_data['structure_type'] == structure]
    axes[1,2].hist(subset['frequency_hz'], alpha=0.6, label=structure, bins=10)
axes[1,2].set_xlabel('Frequency (Hz)')
axes[1,2].set_ylabel('Count')
axes[1,2].set_title('Frequency Distribution by Structure')
axes[1,2].legend()

plt.tight_layout()
plt.show()

# Summary statistics
print("\\n=== SONIFICATION MAPPING SUMMARY ===")
print(f"Frequency range: {frequencies.min():.1f} - {frequencies.max():.1f} Hz")
print(f"Attack time range: {attacks.min():.3f} - {attacks.max():.3f} s")
print(f"Release time range: {releases.min():.3f} - {releases.max():.3f} s")
print(f"Amplitude range: {amplitudes.min():.3f} - {amplitudes.max():.3f}")
print(f"\\nWaveform distribution:")
for wf_type, count in zip(*np.unique(waveforms, return_counts=True)):
    wf_name = ['Sine (α-helix)', 'Sawtooth (β-sheet)', 'Noise (random coil)'][wf_type]
    print(f"  {wf_name}: {count} compounds")

## 6. Sound Generation and Testing

Now let's test the sonification by playing individual compounds and sequences.

In [ ]:
# Test individual compound sonification
def play_compound(compound_index, use_simple=True):
    """
    Play the sonification for a specific compound
    
    Args:
        compound_index (int): Index of compound in the dataframe
        use_simple (bool): Use simple binding script (True) or full binding script (False)
    """
    if compound_index >= len(binding_data):
        print(f"Invalid compound index. Max index is {len(binding_data)-1}")
        return
    
    compound = binding_data.iloc[compound_index]
    
    print(f"\\n=== Playing Compound {compound['compound_id']} ===")
    print(f"Kd: {compound['kd_nm']:.2f} nM")
    print(f"Structure: {compound['structure_type']}")
    print(f"Molecular Weight: {compound['molecular_weight']:.1f} Da")
    print(f"→ Frequency: {compound['frequency_hz']:.1f} Hz")
    print(f"→ Attack: {compound['attack_time']:.3f} s")
    print(f"→ Release: {compound['release_time']:.3f} s")
    print(f"→ Waveform: {['Sine', 'Sawtooth', 'Noise'][compound['waveform_type']]}")
    
    if sc_interface.check_sclang_available():
        if use_simple:
            sc_interface.play_simple_binding(
                freq=compound['frequency_hz'],
                amp=compound['amplitude'],
                attack=compound['attack_time'],
                release=compound['release_time'],
                waveform=compound['waveform_type']
            )
        else:
            sc_interface.play_binding_event(
                kd=compound['kd_nm'],
                kon=compound['kon_m1s1'],
                koff=compound['koff_s1'],
                structure=compound['waveform_type'],
                mw=compound['molecular_weight'],
                amp=compound['amplitude']
            )
    else:
        print("SuperCollider not available - showing parameters only")

# Play a few example compounds
print("Testing individual compound sonification...")
print("\\n" + "="*50)

# Play strongest binder (lowest Kd)
strongest_idx = binding_data['kd_nm'].idxmin()
print(f"\\nSTRONGEST BINDER (lowest Kd):")
play_compound(strongest_idx)

time.sleep(2)  # Wait between sounds

# Play weakest binder (highest Kd)
weakest_idx = binding_data['kd_nm'].idxmax()
print(f"\\nWEAKEST BINDER (highest Kd):")
play_compound(weakest_idx)

time.sleep(2)

# Play a random compound
random_idx = np.random.randint(0, len(binding_data))
print(f"\\nRANDOM COMPOUND:")
play_compound(random_idx)

In [ ]:
# Play sequences of compounds
def play_binding_sequence(sort_by='kd_nm', ascending=True, n_compounds=10, delay=0.8):
    """
    Play a sequence of compounds sorted by a parameter
    
    Args:
        sort_by (str): Parameter to sort by
        ascending (bool): Sort order
        n_compounds (int): Number of compounds to play
        delay (float): Delay between compounds in seconds
    """
    # Sort the data
    sorted_data = binding_data.sort_values(by=sort_by, ascending=ascending).head(n_compounds)
    
    print(f"\\n=== Playing sequence of {n_compounds} compounds ===")
    print(f"Sorted by: {sort_by} ({'ascending' if ascending else 'descending'})")
    print(f"Delay between sounds: {delay}s")
    
    if sc_interface.check_sclang_available():
        sc_interface.play_sequence_from_data(
            frequencies=sorted_data['frequency_hz'].values,
            amplitudes=sorted_data['amplitude'].values,
            attacks=sorted_data['attack_time'].values,
            releases=sorted_data['release_time'].values,
            waveforms=sorted_data['waveform_type'].values,
            delay=delay
        )
        
        # Show the sequence details
        print("\\nSequence details:")
        for i, (_, compound) in enumerate(sorted_data.iterrows()):
            print(f"{i+1:2d}. {compound['compound_id']}: "
                  f"Kd={compound['kd_nm']:6.1f}nM, "
                  f"freq={compound['frequency_hz']:5.1f}Hz, "
                  f"{compound['structure_type']}")
    else:
        print("SuperCollider not available - showing sequence details only")
        for i, (_, compound) in enumerate(sorted_data.iterrows()):
            print(f"{i+1:2d}. {compound['compound_id']}: "
                  f"Kd={compound['kd_nm']:6.1f}nM → {compound['frequency_hz']:5.1f}Hz")

print("\\n" + "="*60)
print("SEQUENCE DEMONSTRATIONS")
print("="*60)

# 1. Play compounds from strongest to weakest binder
print("\\n1. BINDING AFFINITY SEQUENCE (strong → weak)")
print("   Listen for: Low to high pitch (strong binders = low frequency)")
play_binding_sequence(sort_by='kd_nm', ascending=True, n_compounds=8, delay=1.0)

time.sleep(3)

# 2. Play compounds by molecular weight
print("\\n2. MOLECULAR WEIGHT SEQUENCE (light → heavy)")  
print("   Listen for: Quiet to loud (heavy molecules = higher amplitude)")
play_binding_sequence(sort_by='molecular_weight', ascending=True, n_compounds=8, delay=0.6)

time.sleep(3)

# 3. Play compounds by structure type
print("\\n3. STRUCTURE TYPE COMPARISON")
print("   Listen for: Different timbres (sine, sawtooth, noise)")

# Play one of each structure type
for structure in ['alpha_helix', 'beta_sheet', 'random_coil']:
    subset = binding_data[binding_data['structure_type'] == structure]
    if len(subset) > 0:
        example = subset.iloc[0]
        print(f"\\n{structure.upper()}: {example['compound_id']}")
        play_compound(example.name)
        time.sleep(1.5)

## 7. Advanced Analysis and Validation

Let's implement some validation tools to assess the effectiveness of our sonification.

In [ ]:
def analyze_sonification_effectiveness():
    """
    Analyze how well the sonification preserves biological information
    """
    print("=== SONIFICATION EFFECTIVENESS ANALYSIS ===\\n")
    
    # 1. Dynamic range analysis
    print("1. DYNAMIC RANGE ANALYSIS")
    print(f"   Frequency range: {frequencies.max()/frequencies.min():.1f}x dynamic range")
    print(f"   Amplitude range: {amplitudes.max()/amplitudes.min():.1f}x dynamic range")
    print(f"   Attack time range: {attacks.max()/attacks.min():.1f}x dynamic range")
    print(f"   Release time range: {releases.max()/releases.min():.1f}x dynamic range")
    
    # 2. Correlation analysis
    print("\\n2. PARAMETER CORRELATION ANALYSIS")
    correlations = binding_data[['kd_nm', 'frequency_hz', 'molecular_weight', 'amplitude', 
                               'attack_time', 'release_time']].corr()
    
    print(f"   Kd ↔ Frequency correlation: {correlations.loc['kd_nm', 'frequency_hz']:.3f}")
    print(f"   MW ↔ Amplitude correlation: {correlations.loc['molecular_weight', 'amplitude']:.3f}")
    
    # 3. Perceptual distinctiveness
    print("\\n3. PERCEPTUAL DISTINCTIVENESS")
    
    # Calculate frequency differences between structure types
    freq_by_structure = {}
    for structure in ['alpha_helix', 'beta_sheet', 'random_coil']:
        subset = binding_data[binding_data['structure_type'] == structure]
        freq_by_structure[structure] = subset['frequency_hz'].mean()
    
    print("   Average frequencies by structure type:")
    for structure, avg_freq in freq_by_structure.items():
        print(f"     {structure}: {avg_freq:.1f} Hz")
    
    # 4. Information preservation
    print("\\n4. INFORMATION PRESERVATION")
    
    # Check if we can distinguish strong vs weak binders
    strong_binders = binding_data[binding_data['kd_nm'] < binding_data['kd_nm'].median()]
    weak_binders = binding_data[binding_data['kd_nm'] >= binding_data['kd_nm'].median()]
    
    strong_freq_avg = strong_binders['frequency_hz'].mean()
    weak_freq_avg = weak_binders['frequency_hz'].mean()
    
    print(f"   Strong binders (Kd < median): {strong_freq_avg:.1f} Hz average")
    print(f"   Weak binders (Kd ≥ median): {weak_freq_avg:.1f} Hz average")
    print(f"   Frequency separation: {abs(weak_freq_avg - strong_freq_avg):.1f} Hz")
    
    # 5. Recommendations
    print("\\n5. SONIFICATION RECOMMENDATIONS")
    
    freq_separation = abs(weak_freq_avg - strong_freq_avg)
    if freq_separation > 100:
        print("   ✓ Good frequency separation between strong/weak binders")
    else:
        print("   ⚠ Consider increasing frequency range for better discrimination")
    
    amp_range = amplitudes.max() - amplitudes.min()
    if amp_range > 0.3:
        print("   ✓ Good amplitude range for molecular weight representation")
    else:
        print("   ⚠ Consider increasing amplitude range")
    
    return correlations

def create_sonification_summary():
    """Create a summary table of the sonification mappings"""
    
    summary_data = {
        'Biological Parameter': [
            'Binding Affinity (Kd)', 
            'Association Rate (kon)', 
            'Dissociation Rate (koff)',
            'Molecular Weight',
            'Protein Structure'
        ],
        'Musical Parameter': [
            'Pitch/Frequency',
            'Attack Time', 
            'Release Time',
            'Amplitude/Volume',
            'Waveform/Timbre'
        ],
        'Mapping Logic': [
            'Low Kd (strong) → Low pitch (stable)',
            'Fast kon → Short attack (quick onset)',
            'Fast koff → Short release (quick decay)', 
            'Heavy molecules → Loud volume (presence)',
            'Structure type → Waveform character'
        ],
        'Range': [
            f"{frequencies.min():.0f}-{frequencies.max():.0f} Hz",
            f"{attacks.min():.3f}-{attacks.max():.3f} s",
            f"{releases.min():.3f}-{releases.max():.3f} s",
            f"{amplitudes.min():.2f}-{amplitudes.max():.2f}",
            "Sine/Sawtooth/Noise"
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df

# Run the analysis
correlations = analyze_sonification_effectiveness()

print("\\n" + "="*60)
print("SONIFICATION MAPPING SUMMARY TABLE")
print("="*60)

summary_table = create_sonification_summary()
display(summary_table)

# Create a correlation heatmap
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlations, dtype=bool))
sns.heatmap(correlations, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Parameter Correlation Matrix')
plt.tight_layout()
plt.show()

## 8. Next Steps and Extensions

This notebook implements Phase 1 of the molecular music sonification project. Here are the next steps for development:

In [ ]:
print("=== MOLECULAR MUSIC SONIFICATION PROJECT ===")
print("Phase 1: Proof of Concept - COMPLETED ✓")
print()
print("ACHIEVEMENTS:")
print("• ✓ Implemented core parameter mappings (Kd→pitch, kinetics→temporal, etc.)")
print("• ✓ Created sample molecular binding dataset")
print("• ✓ Built SuperCollider integration with external scripts")
print("• ✓ Developed visualization and analysis tools")
print("• ✓ Validated sonification effectiveness")
print()
print("NEXT PHASES:")
print()
print("Phase 2: Real Data Integration (Weeks 3-4)")
print("• Connect to ChEMBL API for real binding data")
print("• Implement PDB structure analysis")
print("• Add more sophisticated kinetic models")
print("• Expand to 100+ compounds for validation")
print()
print("Phase 3: Complex Systems (Weeks 5-8)")
print("• Multi-site cooperative binding")
print("• Enzyme kinetics with Michaelis-Menten")
print("• Competitive and allosteric interactions")
print("• Real-time molecular dynamics sonification")
print()
print("Phase 4: Applications (Weeks 9-12)")
print("• Educational modules for biochemistry courses")
print("• Drug discovery workflow integration")
print("• User studies and perceptual validation")
print("• Publication and open-source release")
print()
print("TECHNICAL EXTENSIONS:")
print("• 3D spatial audio for protein conformations")
print("• Machine learning for pattern recognition")
print("• Web interface for broader accessibility")
print("• MIDI controller integration for real-time manipulation")
print()
print("FILES CREATED:")
print("• molecular_music_sonification.ipynb - Main workflow notebook")
print("• supercollider_scripts/molecular_synthdefs.scd - Core synthesis definitions")
print("• supercollider_scripts/play_binding_event.scd - Single event player")
print("• supercollider_scripts/play_simple_binding.scd - Simplified event player")
print("• supercollider_scripts/play_sequence.scd - Sequence player")
print()
print("USAGE:")
print("1. Ensure SuperCollider is installed and sclang is in PATH")
print("2. Run all notebook cells to generate and analyze data")
print("3. Use play_compound() and play_binding_sequence() functions")
print("4. Modify parameters in mapper class for different mappings")
print("5. Extend with real molecular data from public databases")

# Save the current binding data for future use
binding_data.to_csv('sample_binding_data.csv', index=False)
print("\\n• sample_binding_data.csv - Generated sample dataset saved")

print("\\n" + "="*50)
print("🎵 MOLECULAR MUSIC SONIFICATION READY! 🧬")
print("="*50)